# Querying Data

Topics included in this notebook:  
    
* [Simple queries using pymapd](#pymapd)
* [Introducing Ibis](#ibis)
* [Advanced Ibis Queries](#advanced)

<a id='pymapd'></a>
## Simple queries using `pymapd`

Create a connection. For this example we'll use an external database.

In [ ]:
from pymapd import connect
con = connect(user="mapd", password="HyperInteractive", host="metis.mapd.com", dbname="mapd", port=443, protocol='https')
con 

View available tables in this database

In [ ]:
con.get_tables()

Create a cursor

In [ ]:
c = con.cursor()
c

Query database table of flight departure and arrival delay times

In [ ]:
c.execute("SELECT depdelay, arrdelay FROM flights_donotmodify LIMIT 100")

Display number of rows returned

In [ ]:
c.rowcount

Display the Description objects list

In [ ]:
c.description

Iterate over the cursor, returning a list of tuples of values

In [ ]:
result = list(c)
result[:5]

Close the connection to the database

In [ ]:
con.close()

<a id='ibis'></a>
## Introducing Ibis

In this example, we will be using Ibis to create and manage our connection to the database. Ibis will allow us to construct complex data anlytics using a Pandas-like API. It will convert our analytics methods to a SQL query, but will push the computational burden of the query to the server. In this way, users can query extremely large databases on remote servers without heavy local computation.  
  
For this example we'll use a local database running inside of docker.

Connect to the database using ibis:

In [ ]:
import warnings

from ibis.backends import omniscidb as ibis_omniscidb
    
# set up the credentials to the OmniSci db inside of docker
creds = {
    'user': 'admin',
    'password': 'HyperInteractive',
    'host': '127.0.0.1',
    'port': 16274,
    'dbname': 'omnisci'
}
omnisci_client = ibis_omniscidb.connect(
    user=creds['user'],
    password=creds['password'],
    host=creds['host'],
    port=creds['port'],
    database=creds['dbname']
)

### Exploring the database using Ibis

Let's use the client to take a look at the database (For a more in-depth look at Ibis functionality, check out the Ibis tutorials at [...]() )  
We can quickly get a list of the tables available in the database.  

In [ ]:
omnisci_client.list_tables()

Now we will make a connection to the 'omnisci_states' table.

In [ ]:
states = omnisci_client.table('omnisci_states')
states

You'll notice that when you inspect `states` you see a schema object, not actual results.   
Ibis won't load the data and perform any computation, but instead will leave the data in the   
backend defined in the connection, and we will _ask_ the backend to perform the computations.  

This is a valuable tool when working with big data in which our client side cannot handle the  
volume of data until we have reduced it.  

If you'd perfer for the backend to run the computation immediately, you can set  
`ibis.options.interactive = True`.
  
Let's take a quick look at information Ibis has for this table without actually pulling the data locally:

In [ ]:
# get the table info
display(states.info())

# get the table metadata
display(states.metadata())

Ibis is converting our expression into a SQL expression. Let's take a look at the actual SQL query.

In [ ]:
states.compile()

The table has 52 rows which is small enough for us to handle locally so we can go ahead and execute the query which will bring us back the requested table (we haven't asked it to perform any calculations yet)  

In [ ]:
# execute the query
states_df = states.execute()
print(f'Return Type: {type(states_df)}')
states_df

Now we can immediately continue our data analytics using a Pandas DataFrame (or GeoPandas GeoDataFrame) or we can modify our Ibis query to perform some calculations before pulling back data.  

<a id='advanced'></a>
## Advanced Ibis Queries

We can use Ibis to construct complex SQL queries and push the computation required for these queries to the server. This puts the computational burden on the server rather than the local machine, and allows for users to transform and reduce the dataset before
bringing the results back locally.  
  
For this example we'll use the countries table in our local database.

In [ ]:
countries = omnisci_client.table('omnisci_countries')

Now let's see how many countries are in the Melanesia subregion. First we apply a filter, then we apply a count method.  

In [ ]:
melanesia = countries.filter(countries['subregion'] == 'Melanesia')
melanesia_count = melanesia.count()
type(melanesia_count)

At this point, you can see that we've built up an Ibis expression for this analysis, but we need to tell the server to perform the computation. We do that with the execute statement.

In [ ]:
melanesia_count.execute()

Let's build another complex query using this table. Let's see what the top 5 countries are in terms geographic area.   
  
First, we'll add an additional area column to the dataset. This is calculated from the shapely objects in the `omnisci_geo` column.

In [ ]:
countries = countries.mutate(area=countries.omnisci_geo.area())

We can stop here and perform the computation (e.g. `countries.execute()`), but for this example, we'll continue to build the expression.   
  


In [ ]:
top_five_table = countries.sort_by(('area', False)).head(5)
top_five_countries = top_five_table['admin']
top_five_countries.execute()

Before moving on, we'll close our connection to the database.

In [ ]:
omnisci_client.close()